In [1]:
import tensorflow as tf
import read_stl10_file
from datetime import datetime
import numpy as np

writer = tf.summary.FileWriter("./logs/{}".format(datetime.now().strftime("%Y-%m-%d-%H-%M")))

step = tf.Variable(0, trainable=False)
starter_learning_rate = 0.001
learning_rate = tf.train.exponential_decay(starter_learning_rate, step,
                                                20000, 0.9, staircase=True)
layers = {}


training = tf.placeholder(tf.bool)
X = tf.placeholder(tf.float32, [None, 96, 96, 1], name='X')
Y = tf.placeholder(tf.float32, [None, 10], name='Y')

conv1 = tf.layers.conv2d(inputs=X, filters=64, kernel_size=[3, 3],
                         padding='SAME', activation=tf.nn.relu, name='conv1')
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[3, 3], padding='SAME', strides=3, name='pool1')
dropout1 = tf.layers.dropout(inputs=pool1, rate=0.3, training=training, name='')
layers['conv1'] = conv1
print(pool1)

conv2 = tf.layers.conv2d(inputs=dropout1, filters=64, kernel_size=[3, 3],
                         padding='SAME', activation=tf.nn.relu, name='conv2')
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], padding='SAME', strides=2, name='pool2')
dropout2 = tf.layers.dropout(inputs=pool2, rate=0.3, training=training)
layers['conv2'] = conv2
print(pool2)

conv3 = tf.layers.conv2d(inputs=dropout2, filters=128, kernel_size=[3, 3],
                         padding='SAME', activation=tf.nn.relu, name='conv3')
pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], padding='SAME', strides=2, name='pool3')
dropout3 = tf.layers.dropout(inputs=pool3, rate=0.3, training=training)
layers['conv3'] = conv3
print(pool3)

conv4 = tf.layers.conv2d(inputs=dropout3, filters=256, kernel_size=[3, 3],
                         padding='SAME', activation=tf.nn.relu, name='conv4')
pool4 = tf.layers.max_pooling2d(inputs=conv4, pool_size=[2, 2], padding='SAME', strides=2, name='pool4')
dropout4 = tf.layers.dropout(inputs=pool4, rate=0.3, training=training)
layers['conv4'] = conv4
print(pool4)

flat = tf.reshape(dropout4, [-1, 4 * 4 * 256])
dense5 = tf.layers.dense(inputs=flat, units=625, activation=tf.nn.relu, name='dense5')
dropout5 = tf.layers.dropout(inputs=dense5, rate=0.5, training=training)
print(dense5)
logits = tf.layers.dense(inputs=dropout5, units=10, name='logits')

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y), name='cost')
optimizer = tf.train.AdamOptimizer(
    learning_rate=learning_rate).minimize(cost, global_step=step)

correct_prediction = tf.equal(
    tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='pred')

# tensorboard
cost_summ = tf.summary.scalar("cost", cost)
accuracy_summ = tf.summary.scalar("accuracy", accuracy)
merged_summary = tf.summary.merge_all()

def predict(sess, x_test, training_=False):
    return sess.run(logits,
                         feed_dict={X: x_test, training: training_})

def get_accuracy(sess, x_test, y_test, training_=False):
    return sess.run(accuracy,
                         feed_dict={X: x_test,
                                    Y: y_test, training: training_})

def train(sess, x_data, y_data, training_=True):
    summary, c, _ = sess.run([merged_summary, cost, optimizer], feed_dict={
        X: x_data, Y: y_data, training: training_})
    writer.add_summary(summary)
    return c, _


def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]


def flip_images(X_imgs):
    X_flip = []
    tf.reset_default_graph()
    X = tf.placeholder(tf.float32, shape=(96, 96, 1))
    tf_img1 = tf.image.flip_left_right(X)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for img in X_imgs:
            flipped_imgs = sess.run([tf_img1], feed_dict={X: img})
            X_flip.extend(flipped_imgs)
    X_flip = np.array(X_flip, dtype=np.float32)
    return X_flip


def central_scale_images(X_imgs, scales):
    # Various settings needed for Tensorflow operation
    boxes = np.zeros((len(scales), 4), dtype=np.float32)
    for index, scale in enumerate(scales):
        x1 = y1 = 0.5 - 0.5 * scale  # To scale centrally
        x2 = y2 = 0.5 + 0.5 * scale
        boxes[index] = np.array([y1, x1, y2, x2], dtype=np.float32)
    box_ind = np.zeros((len(scales)), dtype=np.int32)
    crop_size = np.array([96, 96], dtype=np.int32)

    X_scale_data = []
    tf.reset_default_graph()
    X = tf.placeholder(tf.float32, shape=(1, 96, 96, 1))
    # Define Tensorflow operation for all scales but only one base image at a time
    tf_img = tf.image.crop_and_resize(X, boxes, box_ind, crop_size)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        for img_data in X_imgs:
            batch_img = np.expand_dims(img_data, axis=0)
            scaled_imgs = sess.run(tf_img, feed_dict={X: batch_img})
            X_scale_data.extend(scaled_imgs)

    X_scale_data = np.array(X_scale_data, dtype=np.float32)
    return X_scale_data


sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()
writer.add_graph(sess.graph)


# parameter 개수 측정
print("----------parameter count start-----------")
total_parameters = 0
for variable in tf.trainable_variables():
    # shape is an array of tf.Dimension
    shape = variable.get_shape()
    print(shape)
    print(len(shape))
    variable_parameters = 1
    for dim in shape:
        print(dim)
        variable_parameters *= dim.value
    print(variable_parameters)
    total_parameters += variable_parameters
print(total_parameters)
print("----------parameter count end-----------")

# 데이터 불러오기, test set과 train set을 바꿔서 불러온다.
images_test = read_stl10_file.read_all_images(read_stl10_file.DATA_PATH)
labels_test = read_stl10_file.read_labels(read_stl10_file.LABEL_PATH)
labels_test = labels_test - 1  # stl10 data의 index가 1부터 시작하므로, 1을 빼서 0을 start index로 설정.

images_train = read_stl10_file.read_all_images(read_stl10_file.TEST_DATA_PATH)
labels_train = read_stl10_file.read_labels(read_stl10_file.TEST_LABEL_PATH)
labels_train = labels_train - 1  # stl10 data의 index가 1부터 시작하므로, 1을 빼서 0을 start index로 설정.

# data transform
t1 = tf.one_hot(labels_train, depth=10)
t2 = tf.one_hot(labels_test, depth=10)
i1 = tf.image.rgb_to_grayscale(images_train)
i2 = tf.image.rgb_to_grayscale(images_test)
labels_train_onehot, labels_test_onehot, images_train, images_test = sess.run([t1, t2, i1, i2])

# data augmentation 과정
flipped = flip_images(images_train)
centered = central_scale_images(images_train, [0.9])
images_train = np.vstack((images_train, flipped, centered))
labels_train_onehot = np.vstack((labels_train_onehot, labels_train_onehot, labels_train_onehot))
images_train, labels_train_onehot = unison_shuffled_copies(images_train, labels_train_onehot)

# test set을 test set과 validation set으로 분리
images_test_count = images_test.shape[0]
test_valid_point = int(images_test_count * 0.9)

labels_valid_onehot = labels_test_onehot[test_valid_point:]
images_valid = images_test[test_valid_point:]

labels_test_onehot = labels_test_onehot[:test_valid_point]
images_test = images_test[:test_valid_point]

# train, test set의 개수, (8000 * 3, 5000 * 0.9)
images_train_count = images_train.shape[0]
images_test_count = images_test.shape[0]
images_valid_count = images_valid.shape[0]

training_epochs = 60
batch_size = 100

print('Learning Started!')
print("Learning Rate:{}, Epochs: {}, Batch size:{}".format(starter_learning_rate, training_epochs, batch_size))

# train my model
for epoch in range(training_epochs):
    avg_cost = 0
    avg_accuracy = 0
    total_batch = int(images_train_count / batch_size)
    valid_batch = int(images_valid_count / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = images_train[i * batch_size:(i + 1) * batch_size], labels_train_onehot[
                                                                                i * batch_size:(i + 1) * batch_size]
        c, _ = train(sess, batch_xs, batch_ys)
        avg_cost += c / total_batch

    for i in range(valid_batch):
        batch_xs, batch_ys = images_valid[i * batch_size:(i + 1) * batch_size], labels_valid_onehot[
                                                                                i * batch_size:(i + 1) * batch_size]
        acc = get_accuracy(sess, batch_xs, batch_ys)
        avg_accuracy += acc / valid_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost), str(datetime.now()),
          'Validation Set Accuracy:{:.4f}'.format(avg_accuracy))

print('Learning Finished!')

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


sys.version_info(major=3, minor=6, micro=4, releaselevel='final', serial=0)
Tensor("pool1/MaxPool:0", shape=(?, 32, 32, 64), dtype=float32)
Tensor("pool2/MaxPool:0", shape=(?, 16, 16, 64), dtype=float32)
Tensor("pool3/MaxPool:0", shape=(?, 8, 8, 128), dtype=float32)
Tensor("pool4/MaxPool:0", shape=(?, 4, 4, 256), dtype=float32)
Tensor("dense5/Relu:0", shape=(?, 625), dtype=float32)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

----------parameter count start-----------
(3, 3, 1, 64)
4
3
3
1
64
576
(64,)
1
64
64
(3, 3, 64, 64)
4
3
3
64
64
36864
(64,)
1
64
64
(3, 3, 64, 128)
4
3
3
64
128
73728
(128,)
1
128
128
(3, 3, 128, 256)
4
3
3
128
256
294912
(256,)
1
256
256
(4096, 625)
2
4096
625
2560000
(625,)
1
625
625
(625, 10)
2
625
10
6250
(10,)
1
10
10
2973477
----------parameter count end-----------
Learning Started!
Learning Rate:0.001, Epochs: 6

In [3]:
test_batch_size = 1000
total_test_batch = int(images_test_count / test_batch_size)

acc_sum = 0
for i in range(total_test_batch):
    batch_xs, batch_ys = images_test[i*test_batch_size:(i+1)*test_batch_size], labels_test_onehot[i*test_batch_size:(i+1)*test_batch_size]
    acc = get_accuracy(sess, batch_xs, batch_ys)
    print('step', i+1, 'Accuracy:', acc)
    acc_sum += acc

print("Test Accuracy: {:.4f}".format(acc_sum / total_test_batch) )

step 1 Accuracy: 0.688
step 2 Accuracy: 0.695
step 3 Accuracy: 0.699
step 4 Accuracy: 0.725
Test Accuracy: 0.7018


In [4]:
saver.save(sess, './model/final_model')

'./model/final_model'